In [0]:
#requirements packages
%pip install faker
dbutils.library.restartPython()

In [0]:
# %python 

from faker import Faker
import random
import datetime
import json
import os

catalog_name = "dev_catalog"
schema_name = "default"

spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema_name}")

def game_info(n: int , fake: Faker) -> list:
    publisher = [
        'Electronic Arts','Activity Brain','Ubisoft','THQ','Take-Two Interactive','Nintendo','Sony Interactive Entertainment','Microsoft Game Studios','Rockstar Games','Square Enix','Capcom','Bethesda Softworks','Valve','Focus Home Interactive','Kalypso Media','Treyarch','Infinity Ward','BioWare','Paradox Interactive','Bandai Namco Entertainment','Konami','2K Games'
    ]
    game_info = []
    for i in range(n):
        game = {
            "GameID": fake.uuid4(),
            "Publisher": random.choice(publisher),
            "Rating": fake.random_element(elements=['E', 'E10+', 'T', 'M', 'AO', 'RP']),
            "Genre": fake.random_element(elements=['Action', 'Adventure', 'Simulation', 'Strategy', 'RPG', 'Shooter', 'Sports', 'Racing', 'Puzzle', 'Other']),
            "Game_Length" : fake.random_int(min=10, max=100),
            "Game_release_date" : fake.date_between(start_date='-10y', end_date='now').strftime("%Y-%m-%d")
        }
        game_info.append(game)
    return game_info

def game_player_activity(n: int, game_ids: list, fake: Faker) -> list:
    player_activity = []
    for i in range(n):
        start_time = fake.date_time_between(start_date='-2m', end_date='now')
        end_time = fake.date_time_between(start_date=start_time, end_date='now').strftime("%Y-%m-%d %H:%M:%S")
        start_time = start_time.strftime("%Y-%m-%d %H:%M:%S")
        activity = {
            "PlayerID": fake.uuid4(),
            "GameID": random.choice(game_ids),
            "SessionID": fake.uuid4(),
            "Activity_type": fake.random_element(elements=['Play', 'Pause', 'Quit', 'Resume', 'Skip', 'Other','In-queue', 'Watch-trailer', 'Watch-video', 'Watch-ad']),
            "Level": fake.random_int(min=1, max=10),
            "ExperiencePoints": float(fake.random_int(min=100, max=100000)),
            "AchievementUnlocked": fake.random_int(min=0, max=10),
            "CurrencyEarned": fake.pyfloat(left_digits=6, right_digits=2, positive=True),
            "CurrencySpent": fake.pyfloat(left_digits=6, right_digits=2, positive=True),
            "QuestCompleted": fake.random_int(min=0, max=100),
            "EnemiesDefeated": fake.random_int(min=0, max=100),
            "ItemsCollected": fake.random_int(min=0, max=100),
            "Deaths": fake.random_int(min=0, max=10),
            "DistanceTravelled": fake.random_int(min=100, max=100000),
            "ChatMessagesSent": fake.random_int(min=0, max=100),
            "TeamEventsParticipated": fake.random_int(min=0, max=10),
            "PlayMode": fake.random_element(elements=['Singleplayer', 'Multiplayer', 'Co-op', 'Online Co-op', 'Online Multiplayer', 'Online Co-op', 'Online Multiplayer', 'Online Co-op', 'Online'])
        }
        player_activity.append(activity)
    return player_activity

def upload_json_to_landingzone(destination_path: str, filename: str, data: list):
    generated_json = json.dumps(data, indent=4)
    full_path = os.path.join(destination_path, filename)
    dbutils.fs.put(full_path, generated_json, overwrite=True)

def main():
    num_games = 10
    num_player_activity = 10000

    fake = Faker()
    games = game_info(num_games, fake)
    game_player_activities = game_player_activity(num_player_activity, [game['GameID'] for game in games], fake)

    destination_path = f'dbfs:/Volumes/{catalog_name}/{schema_name}/landing_zone/game_analytics'
    game_filename = f"Game/{datetime.datetime.now().strftime('%Y-%m-%d')}/games_{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.json"
    player_activity_filename = f"PlayerActivity/{datetime.datetime.now().strftime('%Y-%m-%d')}/player_activity_{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}.json"

    upload_json_to_landingzone(destination_path, game_filename, games)
    upload_json_to_landingzone(destination_path, player_activity_filename, game_player_activities)

main()